In [ ]:
%uv pip install -U strands-agents-tools strands-agents crewai 

In [14]:
from strands import Agent
from strands.models import BedrockModel
import boto3
from strands_tools import calculator
from pydantic import BaseModel
from crewai.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field
from crewai import Agent, Crew, Task, LLM

In [ ]:
session = boto3.Session()

credentials = session.get_credentials().get_frozen_credentials()
aws_access_key_id = credentials.access_key
aws_secret_access_key = credentials.secret_key
aws_session_token = credentials.token  
aws_region_name = session.region_name

llm = LLM(
    model="bedrock/anthropic.claude-3-sonnet-20240229-v1:0",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    aws_region_name=aws_region_name
)

In [ ]:
class MyToolInput(BaseModel):
    """Input schema for MyCustomTool."""
    argument: str = Field(..., description="The user's mathematical conversion query.")

class StrandsAgentConversionTool(BaseTool):
    name: str = "Strands Agent for mathematical conversions"
    description: str = "This Agent as a tool accurately converts units of measurement, such as converting feet to meters or miles to kilometers, etc."
    args_schema: Type[BaseModel] = MyToolInput

    def _run(self, argument: str) -> str:
        SYSTEM_INSTRUCTION = f"""
            You are a specialized assistant for mathematical conversions. 
            Your sole purpose is to use the 'calculator' tool to answer questions about conversions. 
            If the user asks about anything other than mathematical conversions,
            politely state that you cannot help with that topic and can only assist with math conversion-related queries. 
            Do not attempt to answer unrelated questions or use tools for other purposes.

            Respond in the following format using <answer> tags:
            <answer>
            ANSWER: Your answer here.
            </answer>"""
        
        model = BedrockModel(
            model_id='us.anthropic.claude-3-7-sonnet-20250219-v1:0',
            boto_session=session,
            temperature=0.01,
        )

        agent = Agent(
            system_prompt=SYSTEM_INSTRUCTION, model=model, tools=[calculator]
        )

        result = agent(argument)
        response_text = result.message["content"][0]["text"]
        return response_text

## Testing the tool

In [5]:
tool = StrandsAgentConversionTool()
print(tool.run(argument="Convert 100C to Fahrenheit."))

Using Tool: Strands Agent for mathematical conversions
been called
I'll help you convert 100 degrees Celsius to Fahrenheit using the temperature conversion formula.
Tool #1: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ (100 * 9/5) + 32    │                                                                            │
│  │ Result    │ 212                 │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

<answer>
ANSWER: 100 degrees Celsius is equal to 212 degrees Fahrenheit.

The conversion from Celsius to Fahrenheit uses the formula: °F = (°C × 9/5) + 32
</answer><answer>
ANSWER: 100 degrees Celsius is equal to 212 degrees Fahrenheit.

The conversion from Celsius to Fahrenheit uses the formula: °F = (°C × 9/5) + 32
</answer>


In [ ]:
# Create an agent with Bedrock as the LLM provider
conversion_agent = Agent(
    role="You are a helpful assistant that can answer questions about various things as well as perform mathematical conversions using an agent as a tool.",
    goal="Answer the user's question.",
    backstory="You have access to your pretrained knowledge and can use a specialized agent for mathematical conversions.",
    tools=[StrandsAgentConversionTool()],
    llm=llm
)

In [12]:
task = Task(
  description="Answer the following question: {question}",
  expected_output="A detailed and accurate answer to the user's question.",
  agent=conversion_agent,
)

crew = Crew(
    agents=[conversion_agent],
    tasks=[task],
)

In [13]:
result = crew.kickoff(inputs={"question": "What is 100C in Fahrenheit?"})
print(result)

been called
I'll convert 100 degrees Celsius to Fahrenheit for you.
Tool #3: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ (100 * 9/5) + 32    │                                                                            │
│  │ Result    │ 212                 │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

<answer>
ANSWER: 100 degrees Celsius is equal to 212 degrees Fahrenheit.

The conversion from Celsius to Fahrenheit uses the formula: °F = (°C × 9/5) + 32
</answer>100 degrees Celsius is equal to 212 degrees Fahrenheit.

The conversion from Celsius to Fahrenheit uses the formula: °F = (°C × 9/5) + 32
